In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.inspection import permutation_importance
#from skimpy import skim
import calendar
import holidays
from pandasql import sqldf
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
pio.renderers.default = 'iframe'
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

In [ ]:
df_train=pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
df_store=pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
df_features=pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')

In [ ]:
df_train

In [ ]:
#skim(df_train)

In [ ]:
df_store

In [ ]:
#skim(df_store)

In [ ]:
df_features

In [ ]:
#skim(df_features)

#### So, after observing the 3 datasets i found that:
#### A) Sales (Train)- 
- Date: The date of the week where this observation was taken.
- Weekly_Sales: The sales recorded during that Week.
- Store: The store which observation in recorded 1–45
- Dept: One of 1–99 that shows the department.
- IsHoliday: Boolean value representing a holiday week or not.

#### B) Features-
- Temperature: Temperature of the region during that week.
- Fuel_Price: Fuel Price in that region during that week.
- MarkDown1:5 : Represents the Type of markdown and what quantity was available during that week.
- CPI: Consumer Price Index during that week.
- Unemployment: The unemployment rate during that week in the region of the store.

#### C) Store-
- Store: The store number. Range from 1–45.
- Type: Three types of stores ‘A’, ‘B’ or ‘C’.
- Size: Sets the size of a Store would be calculated by the no. of products available in the particular store ranging from 34,000 to 210,000.

---
## 🍀Merging the datasets Together

### 🍁Merging Sales(Train) & Store

In [ ]:
df_store_sales=pd.merge(df_train, df_store, on='Store')
df_store_sales

### 🍁Merging the Features & Store Sales

In [ ]:
df=pd.merge(df_features, df_store_sales, on=["Store", "Date"])
df

In [ ]:
df.drop('IsHoliday_x', axis=1, inplace=True)

#### 🌟Description of the Features:
- Store: Represents the unique identifier or number assigned to each Walmart store where the sales data was recorded.
- Date: Denotes the specific date when the sales and other associated data were recorded.
- Temperature: Indicates the temperature on the recorded date, which might have an impact on consumer behavior and sales.
- Fuel_Price: Refers to the cost of fuel on the recorded date, which might be relevant as it could affect transportation costs and subsequently impact sales.
- MarkDown1, MarkDown2, MarkDown3, MarkDown4, MarkDown5: These columns might represent promotional markdowns or discounts applied to products in the respective weeks (if available). Markdowns are generally temporary price reductions or promotional offers.
- CPI (Consumer Price Index): Represents the Consumer Price Index, which measures the changes in prices paid by consumers for goods and services. It's an economic indicator that could have an impact on consumer spending.
- Unemployment: Refers to the unemployment rate for the recorded date, which can influence consumer confidence and spending behavior.
- Dept: Represents the department within the Walmart store where the sales were made. Each department might sell different categories of products.
- Weekly_Sales: Indicates the sales amount for a specific department in a store for a particular week or date.
- IsHoliday_y: An indicator denoting whether the recorded date is a holiday.
- Type: Represents the type or category of the Walmart store (e.g., supercenter, discount store, etc.). Different types might have varied sizes, layouts, or product offerings.
- Size: Indicates the size or area of the Walmart store, likely in square footage.

---
## 🍀Data Cleaning & Formating

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.isna().sum()

#### Here we can see that there are actually many missing values for the `Markdown` features. So, right now i'll be skipping thoes columns and perform analysis on the other columns.🙃

In [ ]:
df1=df.copy()

In [ ]:
df1.drop(['MarkDown1', 'MarkDown2', 'MarkDown3', "MarkDown4", "MarkDown5"], axis=1, inplace=True)

In [ ]:
df1

In [ ]:
df1.columns

In [ ]:
df1.info()

In [ ]:
df1.isna().sum()

### 🍁Now we will Manipluate the `Date` Feature.

In [ ]:
df1['Date']=pd.to_datetime(df1['Date'])

In [ ]:
df1['Month'] = df1['Date'].dt.month
df1['Year'] = df1['Date'].dt.year
df1['Month_Name'] = df1['Month'].apply(lambda x: calendar.month_name[x])
df1['Week'] = df1['Date'].dt.isocalendar().week

In [ ]:
df1

In [ ]:
df1.rename(columns={'IsHoliday_y': 'IsHoliday'}, inplace=True)

In [ ]:
df1

In [ ]:
df1.columns

In [ ]:
df1=df1[['Store', 'Dept', 'Type', 'Date', 'Week', 'Month', 'Year',
       'Month_Name', 'IsHoliday', 'Temperature', 'Fuel_Price', 'Unemployment',
       'Size', 'CPI', 'Weekly_Sales']]

In [ ]:
df1

In [ ]:
df1.info()

In [ ]:
df1.to_csv('walmart.csv', index=False)

---
## 🍀Data Analysis

### 🍁Holidays
- Here, we will analyze the week days that the Holidays fall on each year. This is relevant to know how many pre-holiday days are inside each Week marked as `True` inside `IsHoliday` field.
- If, for a certain Week, there are more pre-holiday days in one Year than another, then it is very possible that the Year with more pre-holiday days will have greater Sales for the same Week. So, the model will not take this consideration and we might need to adjust the predicted values at the end.
- Another thing to take into account is that Holiday Weeks but with few or no pre-holiday days might have lower Sales than the Week before.
- We can use SQL, putting the week days for each Holiday in every year. Doing some research, the Super Bowl, Labor Day and Thanksgiving fall on the same day. In the other hand, Christmas is always on December 25th, so the week day can change.hange.hange.

In [ ]:
df1['Week'].unique()

In [ ]:
'''df1=df1
query="""
SELECT
    T.*,
    CASE
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 1 THEN 'Super Bowl'
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 2 THEN 'Labor Day'
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 3 THEN 'Thanksgiving'
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 4 THEN 'Christmas'
    END AS Holiday,
    CASE
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 1 THEN 'Sunday'
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 2 THEN 'Monday'
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 3 THEN 'Thursday'
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 4 AND Year = 2010 THEN 'Saturday'
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 4 AND Year = 2011 THEN 'Sunday'
        WHEN ROW_NUMBER() OVER(PARTITION BY Year ORDER BY Week) = 4 AND Year = 2012 THEN 'Tuesday'
    END AS Day
FROM (
    SELECT DISTINCT
        Year,
        Week
    FROM df1
    WHERE IsHoliday = True
) AS T
"""

result_df=sqldf(query, locals())
result_df
'''

In [ ]:
import holidays
import pandas as pd

def find_holiday_weeks(years, holiday_names):
    for year in years:
        us_holidays = holidays.US(years=year)
        print(f"Holidays in {year}: {list(us_holidays.values())}")
        print(" ")

years = [2010, 2011, 2012]
holiday_names = ['Super Bowl', 'Labor Day', 'Thanksgiving', 'Christmas']

find_holiday_weeks(years, holiday_names)


In [ ]:
import holidays
import pandas as pd

def find_holiday_weeks(years, holiday_names):
    holiday_weeks = []

    for year in years:
        us_holidays = holidays.US(years=year)
        for date, name in us_holidays.items():
            if name in holiday_names:
                week_number = pd.to_datetime(date).isocalendar()[1]
                holiday_weeks.append({'Year': year, 'Holiday': name, 'Week': week_number})

    return pd.DataFrame(holiday_weeks)

years = [2010, 2011, 2012]
holiday_names = ['Super Bowl', 'Labor Day', 'Thanksgiving', 'Christmas Day']

holiday_weeks_df = find_holiday_weeks(years, holiday_names)
holiday_weeks_df


In [ ]:
import datetime

# Get the date of Super Bowl in 2010
super_bowl_date_2010 = datetime.date(2011, 4, 8)

# Calculate the week number using ISO week date
super_bowl_week_2010 = super_bowl_date_2010.isocalendar()[1]
print("Super Bowl in 2010 was in week:", super_bowl_week_2010)


#### 🌟Some interesting notes about the result:
- All Holidays fall on the same week
- Test Data doesn't have Labor Day, so this Holiday is not very relevant.
- Christmas has 0 pre-holiday days in 2010, 1 in 2011 and 3 in 2012. The model will not consider more Sales in 2012 for Test Data, so we are going to adjust it at the end, with a formula and an explanation.

#### 🌟Note:
As we can see, there is one important Holiday not included in `IsHoliday`. It's the `Easter Day`. It is always in a `Sunday`, but can fall on different weeks.
- In 2010 is in Week 13
- In 2011, Week 16
- Week 14 in 2012

In [ ]:
df1.loc[(df1['Year']==2010) & (df1['Week']==13), "IsHoliday"].reset_index()

In [ ]:
df1.loc[(df1['Year']==2011) & (df1['Week']==16), "IsHoliday"].reset_index()

In [ ]:
df1.loc[(df1['Year']==2012) & (df1['Week']==14), "IsHoliday"].reset_index()

In [ ]:
df1.loc[(df1['Year']==2010) & (df1['Week']==13), "IsHoliday"]=True
df1.loc[(df1['Year']==2011) & (df1['Week']==16), "IsHoliday"]=True
df1.loc[(df1['Year']==2012) & (df1['Week']==14), "IsHoliday"]=True

### 🍁Sales

In [ ]:
weekly_sales = df1.groupby('Week')['Weekly_Sales'].sum().reset_index()

fig = px.line(weekly_sales, x='Week', y='Weekly_Sales', title='Total Weekly Sales', markers=True)
fig.update_xaxes(title='Week')
fig.update_yaxes(title='Total Sales')
fig.update_layout(height=400, width=1110)
fig.show()

In [ ]:
years_to_plot = [2010, 2011, 2012]

df_specific_years = df1[df1['Year'].isin(years_to_plot)]

weekly_sales_specific_years = df_specific_years.groupby(['Year', 'Week'])['Weekly_Sales'].sum().reset_index()

fig = px.line(weekly_sales_specific_years, x='Week', y='Weekly_Sales', color='Year',
              title='Total Weekly Sales for Specific Years', markers=True)
fig.update_xaxes(title='Week')
fig.update_yaxes(title='Total Sales')
fig.add_annotation(text="Thanksgiving", x=47, y=70000000, showarrow=False)
fig.add_annotation(text="Christmas", x=51, y=84000000, showarrow=False)
fig.update_layout(height=400, width=1110)
fig.show()

#### 🍂Insight: There's a clear pattern about the sales across the years, by Thanksgiving and Christmas the sales rise up by a huge margin.


In [ ]:
monthly_sales = df1.groupby(['Year', 'Month'])['Weekly_Sales'].sum().reset_index()
years_to_plot = [2010, 2011, 2012]

fig = px.line()
for year in years_to_plot:
    year_data = monthly_sales[monthly_sales['Year'] == year]
    fig.add_scatter(x=year_data['Month'], y=year_data['Weekly_Sales'], mode='lines+markers', name=str(year))

fig.update_layout(title='Monthly Sales for Specific Years', xaxis_title='Month', yaxis_title='Total Sales')
fig.show()

In [ ]:
df1

In [ ]:
store_sales = df1.groupby('Store')['Weekly_Sales'].sum().reset_index()
top_ten_stores = store_sales.nlargest(10, 'Weekly_Sales')
df_top_ten_stores = df1[df1['Store'].isin(top_ten_stores['Store'])]

In [ ]:
top_ten_stores.sort_values(by='Store')

In [ ]:
plt.figure(figsize=(12, 6))
barplot = sns.barplot(data=df_top_ten_stores, x='Store', y='Weekly_Sales', palette='viridis')

bar_width = 0.5  
for i, bar in enumerate(barplot.patches):
    new_x = bar.get_x() + (bar.get_width() / 2) - (bar_width / 2)
    bar.set_x(new_x)
    bar.set_width(bar_width)

plt.title('Total Weekly Sales Comparison for Top Ten Stores')
plt.xlabel('Store')
plt.ylabel('Total Sales')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
sales_2010 = df1[df1['Year'] == 2010]
store_sales_2010 = sales_2010.groupby('Store')['Weekly_Sales'].sum().reset_index()

top_stores_2010 = store_sales_2010.nlargest(10, 'Weekly_Sales')
top_stores_2010.sort_values(by='Store')

In [ ]:
sales_2011 = df1[df1['Year'] == 2011]
store_sales_2011 = sales_2011.groupby('Store')['Weekly_Sales'].sum().reset_index()

top_stores_2011 = store_sales_2011.nlargest(10, 'Weekly_Sales')
top_stores_2011.sort_values(by='Store')

In [ ]:
sales_2012 = df1[df1['Year'] == 2012]
store_sales_2012 = sales_2012.groupby('Store')['Weekly_Sales'].sum().reset_index()

top_stores_2012 = store_sales_2012.nlargest(10, 'Weekly_Sales')
top_stores_2012.sort_values(by='Store')

In [ ]:
monthly_store_sales = df1.groupby(['Store', 'Year'])['Weekly_Sales'].sum().reset_index()
monthly_store_sales

In [ ]:
weekly_store_sales = df1.groupby(['Store', 'Year'])['Weekly_Sales'].sum().reset_index()
store_yearly_sales = weekly_store_sales.groupby(['Store', 'Year'])['Weekly_Sales'].sum().reset_index()
top_stores_by_year = store_yearly_sales.groupby('Year').apply(lambda x: x.nlargest(10, 'Weekly_Sales')).reset_index(drop=True)


palette = ['viridis', 'magma', 'plasma']

plt.figure(figsize=(17, 6))
for idx, year in enumerate(top_stores_by_year['Year'].unique()):
    top_stores_year = top_stores_by_year[top_stores_by_year['Year'] == year]
    ax = plt.subplot(1, len(top_stores_by_year['Year'].unique()), idx + 1)
    sns.barplot(data=top_stores_year, x='Store', y='Weekly_Sales', palette=palette[idx])
    plt.title(f'Top 10 Stores for {year}')
    plt.xlabel('Store')
    plt.ylabel('Total Sales')
    plt.xticks(rotation=0)
    plt.tight_layout()

plt.show()

#### 🍂Insights: The Store 19 was present in the Top 10 Stores during the year 2010, but it was replaced by the Store 39 for the years 2011 & 2012

### 🍁Departments

In [ ]:
df1['Dept'].unique()

In [ ]:
df1['Dept'].unique().size

15, 50, 53, 57, 61-64, 66, 68, 69, 70, 73, 75, 76, 78, 84, 86, 88, 89

In [ ]:
df['Dept'].value_counts().reset_index()

In [ ]:
df1['Weekly_Sales'] = pd.to_numeric(df1['Weekly_Sales'], errors='coerce')
depts = df1.groupby('Dept')['Weekly_Sales'].mean().sort_values(ascending=False)

bar = px.bar(depts, x=depts.index, y=depts.values, title='Department and Sales', color=depts.values)
bar.update_layout(barmode='group', xaxis={'categoryorder': 'total descending'})
bar.update_yaxes(title='Weekly Sales')
bar.update_coloraxes(colorbar_title='Weekly Sales')
bar.show()

#### 🍂Insights: There are some departments that contribute more in the weekly sales, the departments 38, 92, 95 are the highest contributors.

In [ ]:
year = 2010
dept_sales = df1.groupby(['Store', 'Year', 'Dept'])['Weekly_Sales'].sum().reset_index()
top_10_stores_year = top_stores_by_year[top_stores_by_year['Year'] == year]['Store'].tolist()
dept_sales_top_stores_year = dept_sales[(dept_sales['Year'] == year) & (dept_sales['Store'].isin(top_10_stores_year))]

rows, cols = 5, 2

fig = make_subplots(rows=rows, cols=cols, subplot_titles=top_10_stores_year, shared_xaxes=True, shared_yaxes=True)
row, col = 1, 1
for store in top_10_stores_year:
    dept_sales_store_year = dept_sales_top_stores_year[dept_sales_top_stores_year['Store'] == store]
    fig.add_trace(go.Bar(x=dept_sales_store_year['Dept'], y=dept_sales_store_year['Weekly_Sales'], name=store), row=row, col=col)
    col += 1
    if col > cols:
        col = 1
        row += 1

fig.update_layout(title=f'Departmental Sales for Top 10 Stores in {year}', showlegend=False,
                  height=1000, width=1110, title_font_size=24, margin=dict(l=40, r=40, t=80, b=80))

fig.update_xaxes(title_text='Department', row=rows, col=1)
fig.update_xaxes(title_text='Department', row=rows, col=2)
fig.update_yaxes(title_text='Weekly Sales', row=(rows // 2), col=1)
fig.show()

In [ ]:
year = 2011
dept_sales = df1.groupby(['Store', 'Year', 'Dept'])['Weekly_Sales'].sum().reset_index()
top_10_stores_year = top_stores_by_year[top_stores_by_year['Year'] == year]['Store'].tolist()
dept_sales_top_stores_year = dept_sales[(dept_sales['Year'] == year) & (dept_sales['Store'].isin(top_10_stores_year))]

rows, cols = 5, 2

fig = make_subplots(rows=rows, cols=cols, subplot_titles=top_10_stores_year, shared_xaxes=True, shared_yaxes=True)
row, col = 1, 1
for store in top_10_stores_year:
    dept_sales_store_year = dept_sales_top_stores_year[dept_sales_top_stores_year['Store'] == store]
    fig.add_trace(go.Bar(x=dept_sales_store_year['Dept'], y=dept_sales_store_year['Weekly_Sales'], name=store), row=row, col=col)
    col += 1
    if col > cols:
        col = 1
        row += 1

fig.update_layout(title=f'Departmental Sales for Top 10 Stores in {year}', showlegend=False,
                  height=1000, width=1110, title_font_size=24, margin=dict(l=40, r=40, t=80, b=80))

fig.update_xaxes(title_text='Department', row=rows, col=1)
fig.update_xaxes(title_text='Department', row=rows, col=2)
fig.update_yaxes(title_text='Weekly Sales', row=(rows // 2), col=1)
fig.show()

In [ ]:
year = 2012
dept_sales = df1.groupby(['Store', 'Year', 'Dept'])['Weekly_Sales'].sum().reset_index()
top_10_stores_year = top_stores_by_year[top_stores_by_year['Year'] == year]['Store'].tolist()
dept_sales_top_stores_year = dept_sales[(dept_sales['Year'] == year) & (dept_sales['Store'].isin(top_10_stores_year))]

rows, cols = 5, 2

fig = make_subplots(rows=rows, cols=cols, subplot_titles=top_10_stores_year, shared_xaxes=True, shared_yaxes=True)
row, col = 1, 1
for store in top_10_stores_year:
    dept_sales_store_year = dept_sales_top_stores_year[dept_sales_top_stores_year['Store'] == store]
    fig.add_trace(go.Bar(x=dept_sales_store_year['Dept'], y=dept_sales_store_year['Weekly_Sales'], name=store), row=row, col=col)
    col += 1
    if col > cols:
        col = 1
        row += 1

fig.update_layout(title=f'Departmental Sales for Top 10 Stores in {year}', showlegend=False,
                  height=1000, width=1110, title_font_size=24, margin=dict(l=40, r=40, t=80, b=80))

fig.update_xaxes(title_text='Department', row=rows, col=1)
fig.update_xaxes(title_text='Department', row=rows, col=2)
fig.update_yaxes(title_text='Weekly Sales', row=(rows // 2), col=1)
fig.show()

In [ ]:
comparison_df = df1[(df1['Store'].isin([19, 39])) & (df1['Year'] == 2010)]
dept_sales_comparison = comparison_df.groupby(['Store', 'Year', 'Dept'])['Weekly_Sales'].sum().reset_index()
dept_sales_19_39 = dept_sales_comparison[(dept_sales_comparison['Store'].isin([19, 39]))]

pivoted_df = dept_sales_19_39.pivot(index='Dept', columns='Store', values='Weekly_Sales').reset_index()
fig = px.bar(pivoted_df, x='Dept', y=[19, 39], barmode='group',
             labels={'Dept': 'Department', 'value': 'Weekly Sales', 'variable': 'Store'},
             title='Department Sales Comparison for Stores 19 and 39 in 2010')
fig.update_layout(xaxis_title='Department', yaxis_title='Weekly Sales', showlegend=True,
                  width=1100, height=400)
fig.show()

In [ ]:
comparison_df = df1[(df1['Store'].isin([19, 39])) & (df1['Year'] == 2011)]
dept_sales_comparison = comparison_df.groupby(['Store', 'Year', 'Dept'])['Weekly_Sales'].sum().reset_index()
dept_sales_19_39 = dept_sales_comparison[(dept_sales_comparison['Store'].isin([19, 39]))]

pivoted_df = dept_sales_19_39.pivot(index='Dept', columns='Store', values='Weekly_Sales').reset_index()
fig = px.bar(pivoted_df, x='Dept', y=[19, 39], barmode='group',
             labels={'Dept': 'Department', 'value': 'Weekly Sales', 'variable': 'Store'},
             title='Department Sales Comparison for Stores 19 and 39 in 2011')
fig.update_layout(xaxis_title='Department', yaxis_title='Weekly Sales', showlegend=True,
                  width=1100, height=400)
fig.show()

In [ ]:
store_19_sales = df1[(df1['Store'] == 19) & (df1['Year'].isin([2010, 2011]))]
store_19_weekly_sales = store_19_sales.groupby(['Year', 'Week'])['Weekly_Sales'].sum().reset_index()

fig = px.line(store_19_weekly_sales, x='Week', y='Weekly_Sales', color='Year',
              labels={'Week': 'Week', 'Weekly_Sales': 'Total Sales', 'Year': 'Year'},
              title='Store 19 Sales Comparison for 2010 and 2011 by Week')
fig.update_traces(mode='markers+lines')
fig.add_annotation(text="Thanksgiving", x=47, y=2200000, showarrow=False)
fig.add_annotation(text="Christmas", x=51, y=2900000, showarrow=False)
fig.update_layout(xaxis_title='Week', yaxis_title='Total Sales', showlegend=True)
fig.show()

In [ ]:
store_19_sales = df1[(df1['Store'] == 19) & (df1['Year'].isin([2010, 2011]))]
store_19_monthly_sales = store_19_sales.groupby(['Year', 'Month'])['Weekly_Sales'].sum().reset_index()

fig = px.line(store_19_monthly_sales, x='Month', y='Weekly_Sales', color='Year',
              labels={'Month': 'Month', 'Weekly_Sales': 'Total Sales', 'Year': 'Year'},
              title='Store 19 Sales Comparison for 2010 and 2011 by Month')
fig.update_traces(mode='markers+lines')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales', showlegend=True)
fig.show()

In [ ]:
store_19_sales = df1[(df1['Store'] == 39) & (df1['Year'].isin([2010, 2011]))]
store_19_weekly_sales = store_19_sales.groupby(['Year', 'Week'])['Weekly_Sales'].sum().reset_index()

fig = px.line(store_19_weekly_sales, x='Week', y='Weekly_Sales', color='Year',
              labels={'Week': 'Week', 'Weekly_Sales': 'Total Sales', 'Year': 'Year'},
              title='Store 39 Sales Comparison for 2010 and 2011 by Week')
fig.update_traces(mode='markers+lines')
fig.add_annotation(text="Thanksgiving", x=47, y=2500000, showarrow=False)
fig.add_annotation(text="Christmas", x=51, y=2700000, showarrow=False)
fig.update_layout(xaxis_title='Week', yaxis_title='Total Sales', showlegend=True)
fig.show()

In [ ]:
store_19_sales = df1[(df1['Store'] == 39) & (df1['Year'].isin([2010, 2011]))]
store_19_monthly_sales = store_19_sales.groupby(['Year', 'Month'])['Weekly_Sales'].sum().reset_index()
fig = px.line(store_19_monthly_sales, x='Month', y='Weekly_Sales', color='Year',
              labels={'Month': 'Month', 'Weekly_Sales': 'Total Sales', 'Year': 'Year'},
              title='Store 39 Sales Comparison for 2010 and 2011 by Month')
fig.update_traces(mode='markers+lines')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales', showlegend=True)
fig.show()

In [ ]:
stores_19_39_2010 = df1[(df1['Year'] == 2010) & (df1['Store'].isin([19, 39]))]
stores_19_39_2010_weekly_sales = stores_19_39_2010.groupby(['Store', 'Year', 'Week'])['Weekly_Sales'].sum().reset_index()

fig = px.line(stores_19_39_2010_weekly_sales, x='Week', y='Weekly_Sales', color='Store',
              labels={'Week': 'Week', 'Weekly_Sales': 'Total Sales', 'Store': 'Store'},
              title='Stores 19 and 39 Sales Comparison for 2010 by Week')
fig.update_traces(mode='markers+lines')
fig.add_annotation(text="Thanksgiving", x=47, y=2300000, showarrow=False)
fig.add_annotation(text="Christmas", x=51, y=2900000, showarrow=False)
fig.update_layout(xaxis_title='Week', yaxis_title='Total Sales', showlegend=True)
fig.show()

In [ ]:
stores_19_39_2011 = df1[(df1['Year'] == 2010) & (df1['Store'].isin([19, 39]))]
stores_19_39_2011_monthly_sales = stores_19_39_2011.groupby(['Store', 'Year', 'Month'])['Weekly_Sales'].sum().reset_index()

fig = px.line(stores_19_39_2011_monthly_sales, x='Month', y='Weekly_Sales', color='Store',
              labels={'Month': 'Month', 'Weekly_Sales': 'Total Sales', 'Store': 'Store'},
              title='Stores 19 and 39 Sales Comparison for 2010 by Month')
fig.update_traces(mode='markers+lines')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales', showlegend=True)
fig.show()

In [ ]:
stores_19_39_2010 = df1[(df1['Year'] == 2011) & (df1['Store'].isin([19, 39]))]
stores_19_39_2010_weekly_sales = stores_19_39_2010.groupby(['Store', 'Year', 'Week'])['Weekly_Sales'].sum().reset_index()

fig = px.line(stores_19_39_2010_weekly_sales, x='Week', y='Weekly_Sales', color='Store',
              labels={'Week': 'Week', 'Weekly_Sales': 'Total Sales', 'Store': 'Store'},
              title='Stores 19 and 39 Sales Comparison for 2010 by Week')
fig.update_traces(mode='markers+lines')
fig.add_annotation(text="Thanksgiving", x=47, y=2500000, showarrow=False)
fig.add_annotation(text="Christmas", x=51, y=2700000, showarrow=False)
fig.update_layout(xaxis_title='Week', yaxis_title='Total Sales', showlegend=True)
fig.show()

In [ ]:
stores_19_39_2011 = df1[(df1['Year'] == 2011) & (df1['Store'].isin([19, 39]))]
stores_19_39_2011_monthly_sales = stores_19_39_2011.groupby(['Store', 'Year', 'Month'])['Weekly_Sales'].sum().reset_index()

fig = px.line(stores_19_39_2011_monthly_sales, x='Month', y='Weekly_Sales', color='Store',
              labels={'Month': 'Month', 'Weekly_Sales': 'Total Sales', 'Store': 'Store'},
              title='Stores 19 and 39 Sales Comparison for 2011 by Month')
fig.update_traces(mode='markers+lines')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales', showlegend=True)
fig.show()

#### 🍂Insights: We can see that there is significant difference in the performance of the stores 19 and 39 from the month `8` to `12`, also we can clearly see the difference in sales during the week `47` i.e during `Thanksgiving`.This might have resulted in the store 19 to loose it's position from the top 10 stores for the year `2011`.

### 🍁Temperature

In [ ]:
df1['Temperature'].unique()

In [ ]:
df1['Temperature'].unique().size

##### Converting The temperature to `Celsius` for a better interpretation

In [ ]:
df1['Temperature']=df1['Temperature'].apply(lambda x :  (x - 32) / 1.8)

In [ ]:
df1['Temperature'].unique()

In [ ]:
fig = px.histogram(df1, x='Temperature', y='Weekly_Sales', color='IsHoliday', marginal='box', opacity=0.6,
                   title='Temperature and Sales by Holiday', color_discrete_sequence=['orchid', 'darkmagenta'])
fig.show()

In [ ]:
scatter_fig = px.scatter(df1, x='Temperature', y='Weekly_Sales', title='Temperature vs Weekly Sales', color_discrete_sequence=['mediumvioletred'])
scatter_fig.update_layout(xaxis_title='Temperature', yaxis_title='Weekly Sales')
scatter_fig.show()

#### 🍂Insights: There is a pattern between the coldness and the sales, obviously related to the fact that the USA is in the northern hemisphere, and it is a country where a large part of it experiences cold temperatures for this time.

### 🍁Fuel Price

In [ ]:
fig = px.histogram(df1, x='Fuel_Price', y='Weekly_Sales', color='IsHoliday', marginal='box', opacity=0.6,
                   title='Fule Price and Sales by Holiday', color_discrete_sequence=['black', 'gold'])
fig.show()

In [ ]:
scatter_fig = px.scatter(df1, x='Fuel_Price', y='Weekly_Sales', title='Fuel Price vs Weekly Sales', color_discrete_sequence=['firebrick'])
scatter_fig.update_layout(xaxis_title='Fuel Price', yaxis_title='Weekly Sales')
scatter_fig.show()

#### 🍂Insights: There is not a very clear pattern here, but you can see that the lower the fuel price, the more sales.

### 🍁CPI

In [ ]:
fig = px.histogram(df1, x='CPI', y='Weekly_Sales', color='IsHoliday', marginal='box', opacity=0.6,
                   title='CPI and Sales by Holiday', color_discrete_sequence=['mediumseagreen', 'yellowgreen'])
fig.show()

In [ ]:
scatter_fig = px.scatter(df1, x='CPI', y='Weekly_Sales', title='CPI vs Weekly Sales', color_discrete_sequence=['teal'])
scatter_fig.update_layout(xaxis_title='CPI', yaxis_title='Weekly Sales')
scatter_fig.show()

#### 🍂Insights: The CPI indeed affects the Sales behaviour, but not in magnitude i think, also the sales with a middle CPI are lower.

### 🍁Unemployment

In [ ]:
fig = px.histogram(df1, x='Unemployment', y='Weekly_Sales', color='IsHoliday', marginal='box', opacity=0.6,
                   title='Unemployment and Sales by Holiday', color_discrete_sequence=['darkslategrey', 'lightslategrey'])
fig.show()

In [ ]:
scatter_fig = px.scatter(df1, x='Unemployment', y='Weekly_Sales', title='Unemployment vs Weekly Sales', color_discrete_sequence=['gray'])
scatter_fig.update_layout(xaxis_title='Unemployment', yaxis_title='Weekly Sales')
scatter_fig.show()

#### 🍂Insights: It can be seen that the lower the value, higher are the sales, as it makes sense

### 🍁Size & Type

In [ ]:
sizes = df1.groupby('Size')['Weekly_Sales'].sum().reset_index()

# Plotting the aggregated data
fig = px.line(sizes, x='Size', y='Weekly_Sales', title='Store size and Sales')
fig.show()

In [ ]:
plt.figure(figsize=(12, 6)) 
sns.kdeplot(df1['Size'], fill=True)
plt.xlabel('Size')
plt.ylabel('Density')
plt.title('Density Plot of Store Sizes')
plt.show()

fig = px.histogram(df1, x='Size', marginal='rug', nbins=20, histnorm='density')
fig.update_traces(opacity=0.5)
fig.update_layout(
    xaxis_title='Size',
    yaxis_title='Density',
    title='Density Plot of Store Sizes'
)
fig.show()

#### 🍂Insights: Size is an important factor when it comes to sales, as you can see here.

In [ ]:
palletes = {
   'continuos':{'blues': ['#03045E', '#023E8A', '#0077B6', '#0077B6', '#0096C7', '#00B4D8', '#48CAE4', '#90E0EF', '#ADE8F4', '#CAF0F8'],
                'green_n_blues': ['#D9ED92', '#B5E48C', '#99D98C', '#76C893', '#52B69A', '#34A0A4', '#168AAD', '#1A759F', '#1E6091', '#184E77']
               }
}

In [ ]:
px.box(df1, x='Type', y='Size', color='Type',
       title='Store size and Store type', 
             color_discrete_sequence=palletes['continuos']['blues'])

#### 🍂Insights: Within size we can see that there are 3 types of stores, the A are the highest.

In [ ]:
px.box(df1, x='Type', y='Weekly_Sales', color='Type', 
       title='Store type and sales', color_discrete_sequence=palletes['continuos']['blues'])

#### 🍂Insights: In relation to the type of store we can see that although the C are the smallest ones, they are those that have the highest median sales.

In [ ]:
Q1 = df1['Weekly_Sales'].quantile(0.25)
Q3 = df1['Weekly_Sales'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

filtered_data = df1[(df1['Weekly_Sales'] > lower_bound) & (df1['Weekly_Sales'] < upper_bound)]

fig = px.box(filtered_data, x='Type', y='Weekly_Sales', color='Type', 
             title='Store Type and Sales Comparison (Outliers Removed)', color_discrete_sequence=palletes['continuos']['blues'])
fig.update_layout(xaxis_title='Store Type', yaxis_title='Weekly Sales')
fig.show()

In [ ]:
df_iqr_out=df1.loc[(df1['Weekly_Sales']>upper_bound) | (df1['Weekly_Sales']<lower_bound)]
df_iqr_out.shape

### 🍁Correlation with Sales

In [ ]:
df1c=df1.copy()
type_mapping = {'A': 0, 'B': 1, 'C': 2}
df1c['Type_Encoded'] = df1c['Type'].map(type_mapping)
df1c=df1c.select_dtypes(exclude='object')
corr = df1c.corr()

mask = np.triu(np.ones_like(corr, dtype=bool))
df_mask = corr.mask(mask).round(2)

fig = ff.create_annotated_heatmap(
    z=df_mask.to_numpy(),
    x=df_mask.columns.tolist(),
    y=df_mask.columns.tolist(),
    colorscale=palletes['continuos']['green_n_blues'],
    hoverinfo='none',
    showscale=True,
    ygap=1,
    xgap=1
)

fig.update_xaxes(side='bottom')

fig.update_layout(
    title='<b>Feature Correlation (Heatmap)</b>',
    title_x=0.5,
    width=1100,
    height=700,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    yaxis_autorange='reversed',
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.show()

In [ ]:
weekly_sales_corr = df1c.corr().iloc[12, :]
corr_df = pd.DataFrame(data=weekly_sales_corr, index=weekly_sales_corr.index).sort_values(by=['Weekly_Sales'], ascending=False)
corr_df = corr_df.iloc[1:]

fig = px.bar(corr_df, x=corr_df.index, y='Weekly_Sales', color=corr_df.index, labels={'index': 'Features'},
             color_discrete_sequence=palletes['continuos']['green_n_blues'])  # Change the color scheme if needed

fig.update_traces(showlegend=False)
fig.update_layout(width=1100, height=700)
fig.show()

## 🍀Feature Importance

In [ ]:
type_mapping = {True: 0, False: 1}
df1c['IsHoliday_Encoded'] = df1c['IsHoliday'].map(type_mapping)

In [ ]:
df1c.drop('IsHoliday', axis=1, inplace=True)

In [ ]:
df1c=df1c[['Store', 'Dept', 'Week', 'Month', 'Year', 'Temperature', 'Fuel_Price',
       'Unemployment','IsHoliday_Encoded', 'Type_Encoded', 'Size', 'CPI', 'Weekly_Sales'
       ]]

In [ ]:
df1c_sample=df1c.copy().sample(frac=.25)
X=df1c_sample.iloc[:,:-1].values
y=df1c_sample.iloc[:,-1].values
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
feat_model=xgb.XGBRegressor(random_state=0).fit(X_train, y_train)
perm_importance = permutation_importance(feat_model, X_test, y_test, random_state=1)
importances = perm_importance.importances_mean

In [ ]:
importance_df = pd.DataFrame({'Feature': ['Store', 'Dept', 'Week', 'Month', 'Year', 'Temperature', 'Fuel_Price',
       'Unemployment','IsHoliday_Encoded', 'Type_Encoded', 'Size', 'CPI'
       ], 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df

In [ ]:
fig = px.bar(importance_df, y='Importance', x='Feature',
             labels={'Feature': 'Features', 'Importance': 'Importance'},
             orientation='v', title='Feature Importances')

fig.update_traces(marker_color=palletes['continuos']['blues'])
fig.update_layout(width=1100, height=600)
fig.update_layout(yaxis_tickangle=0, yaxis=dict(categoryorder='total descending'))
fig.show()